<h1 id="ML-with-sklearn:-10-models">ML with sklearn: 10 models</h1>

<p>Modle:</p>

<ol>
	<li>Random Forest,</li>
	<li>SVM,</li>
	<li>Logistic,</li>
	<li>Guassian Naive Bayesian,</li>
	<li>Decision Tree,</li>
	<li>KNN,</li>
	<li>AdaBoost,</li>
	<li>XGBoost,</li>
	<li>CatBoost</li>
	<li>DNN</li>
</ol>

<p>Data: kaggle titanic&nbsp;</p>


## Setup

In [101]:
%matplotlib inline

import math, time, random, datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sns

import missingno


# Machine learning pack 1


from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier

from sklearn.svm import SVC
from xgboost import XGBClassifier
import xgboost as xgb

import catboost
from catboost import CatBoostClassifier, Pool, cv

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from mlxtend.classifier import StackingClassifier

#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans, DBSCAN

# Neural networks
#import tensorflow as  tf
#import keras

#from keras.models import Sequential
#from keras.utils import np_utils
#from keras import layers
#from keras import regularizers

#from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')


## Loading data



In [5]:
df_train = pd.read_csv('data/df_train_dp.csv')
df_test = pd.read_csv('data/df_test_dp.csv')
gender_submission = pd.read_csv('data/gender_submission.csv')

df_train.head()

,PassengerId,Survived,SibSp,Parch,Isfemale,Age_fill,Ischild,Fare_fill,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0,1,0.0,0,0.741556,0,0.225126,0,0,1,0,0,1
1,2,1,1,0.0,1,1.291367,0,2.213478,1,0,0,1,0,0
2,3,1,0,0.0,1,0.879009,0,0.246086,0,0,1,0,0,1
3,4,1,1,0.0,1,1.188277,0,1.648853,0,0,1,1,0,0
4,5,0,0,0.0,0,1.188277,0,0.249967,0,0,1,0,0,1


In [6]:
df_test.head()

,PassengerId,SibSp,Parch,Isfemale,Age_fill,Ischild,Fare_fill,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,892,0,0.00000,0,1.175411,0,0.220206,0,1,0,0,0,1
1,893,1,0.00000,1,1.603394,0,0.196884,0,0,1,0,0,1
2,894,0,0.00000,0,2.116973,0,0.272473,0,1,0,0,1,0
3,895,0,0.00000,0,0.918622,0,0.243644,0,0,1,0,0,1
4,896,1,2.54878,1,0.747429,0,0.345602,0,0,1,0,0,1


## Prepare for ML

In [7]:
X_train = np.array(df_train.drop(['Survived', 'PassengerId'], axis=1))
y_train = np.array(df_train['Survived'])

X_test = np.array(df_test.drop(['PassengerId'], axis=1))

In [17]:
X_train.shape[1]

12

In [18]:
y_train.shape

(891,)

In [9]:
X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train, y_train, test_size=0.1,random_state=0)

## Hyper tuning

In [10]:
n_cv = 10

In [11]:
def get_cv_score(classifier_name, n_cv):
    score = [0] * n_cv
    for i in range(0, n_cv):
        command_str = classifier_name + '.cv_results_[\'split' + str(i) + '_test_score\']'
        score[i] = eval(command_str)[0]
    return score


def hyper_tune(model, name, hyper, n_cv, X, y):
    start_time = time.time()

    model.fit(X, y)

    print("training time:", time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time)))

    score_list= get_cv_score(name, n_cv)
    print("Accuracy: {0:.4f} +/- {1:.4f}".format(np.mean(score_list), np.std(score_list)))
    
    opt_param = model.best_estimator_.get_params()
    print("optimized parameters:", {k:opt_param[k] for k in hyper.keys()})
    
    return model, score_list, opt_param

### Random Forest

In [60]:
hyper_rf = dict(
    n_estimators = [60,70,80], # 70
    max_depth = [4,5,6],  # 5
    random_state = [0]
)

gscv_rf = GridSearchCV(RandomForestClassifier(), hyper_rf, scoring = 'accuracy', cv = n_cv)

gscv_out_rf, score_rf, params_rf = hyper_tune(gscv_rf, 'gscv_rf', hyper_rf, n_cv, X_train_.copy(), y_train_.copy())


training time: 00:00:06
Accuracy: 0.8278 +/- 0.0246
optimized parameters: {'n_estimators': 70, 'max_depth': 5, 'random_state': 0}


### SVM

In [61]:
hyper_svm = dict(
    C = [5.0,6.0, 7.0], # 6
    kernel = ['linear', 'poly', 'rbf', 'sigmoid'],  #rbf
    gamma = ['auto', 'scale'], # auto
    probability = [True],
    random_state = [0]
)

gscv_svm = GridSearchCV(SVC(), hyper_svm, scoring = 'accuracy', cv = n_cv)

gscv_out_svm, score_svm, params_svm = hyper_tune(gscv_svm, 'gscv_svm', hyper_svm, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:32
Accuracy: 0.8005 +/- 0.0460
optimized parameters: {'C': 6.0, 'kernel': 'rbf', 'gamma': 'auto', 'probability': True, 'random_state': 0}


### Logistic regression

In [62]:
hyper_logis = dict(
    C = [0.1, 1.0, 10.0], # 1 
    solver = ['lbfgs'],
    random_state = [0]
)

gscv_logis = GridSearchCV(LogisticRegression(), hyper_logis, scoring = 'accuracy', cv = n_cv)

gscv_out_logis, score_logis, params_logis = hyper_tune(gscv_logis, 'gscv_logis', hyper_logis, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:00
Accuracy: 0.8067 +/- 0.0389
optimized parameters: {'C': 1.0, 'solver': 'lbfgs', 'random_state': 0}


### Guassian Naive Bayesian

In [63]:
hyper_gnb = dict(
    var_smoothing = [0,1e-10, 1e-9] # 
    
)

gscv_gnb = GridSearchCV(GaussianNB(), hyper_gnb, scoring = 'accuracy', cv = n_cv)

gscv_out_gnb, score_gnb, params_gnb = hyper_tune(gscv_gnb, 'gscv_gnb', hyper_gnb, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:00
Accuracy: 0.8041 +/- 0.0307
optimized parameters: {'var_smoothing': 0}


### Decistion Tree

In [90]:
hyper_dt = dict(
    max_depth = [4,6,8,10], #
    min_samples_split=[2,10,20],
    min_samples_leaf=[1,10,20],
    max_features = ['sqrt','log2', None]
)

gscv_dt = GridSearchCV(DecisionTreeClassifier(), hyper_dt, scoring = 'accuracy', cv = n_cv)

gscv_out_dt, score_dt, params_dt = hyper_tune(gscv_dt, 'gscv_dt', hyper_dt, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:01
Accuracy: 0.7878 +/- 0.0528
optimized parameters: {'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': None}


### K-Nearest Neighbors

In [65]:
hyper_knn = dict(
    n_neighbors = [5,7,9,11], # 
    algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
)

gscv_knn = GridSearchCV(KNeighborsClassifier(), hyper_knn, scoring = 'accuracy', cv = n_cv)

gscv_out_knn, score_knn, params_knn = hyper_tune(gscv_knn, 'gscv_knn', hyper_knn, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:00
Accuracy: 0.7977 +/- 0.0358
optimized parameters: {'n_neighbors': 9, 'algorithm': 'auto'}


### AdaBoost

In [66]:
hyper_adab = dict(
    n_estimators = [40, 50, 60],# 50
    learning_rate = [0.2, 0.3, 0.4], # 0.3
    random_state = [0]
)c

gscv_adab = GridSearchCV(AdaBoostClassifier(), hyper_adab, scoring = 'accuracy', cv = n_cv)

gscv_out_adab, score_adab, params_adab = hyper_tune(gscv_adab, 'gscv_adab', hyper_adab, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:06
Accuracy: 0.8030 +/- 0.0426
optimized parameters: {'n_estimators': 50, 'learning_rate': 0.3, 'random_state': 0}


### XGBoost

In [67]:
hyper_xgb = dict(
    eta = [0, 1e-5], # 0 
    gamma =[0,1e-5], # 0
    max_depth = [4, 5, 6], # 5
    probability = [True],
    random_state = [0]
)

gscv_xgb = GridSearchCV(XGBClassifier(), hyper_xgb, scoring = 'accuracy', cv = n_cv)

gscv_out_xgb, score_xgb, params_xgb = hyper_tune(gscv_xgb, 'gscv_xgb', hyper_xgb, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:00:10
Accuracy: 0.8290 +/- 0.0214
optimized parameters: {'eta': 0, 'gamma': 0, 'max_depth': 5, 'probability': True, 'random_state': 0}


### CatBoost

In [70]:
hyper_catb = dict(
    depth = [6,8], # 6 
    learning_rate = [0.02], #
    loss_function = ['Logloss'],
    random_state = [0]
)

gscv_catb = GridSearchCV(CatBoostClassifier(iterations = 200,verbose=0), hyper_catb, scoring = 'accuracy', cv = n_cv)

gscv_out_catb, score_carb, params_carb  = hyper_tune(gscv_catb, 'gscv_catb', hyper_catb, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:01:41
Accuracy: 0.8241 +/- 0.0245
optimized parameters: {'depth': 6, 'learning_rate': 0.02, 'loss_function': 'Logloss', 'random_state': 0}


### DNN

In [91]:
hyper_dnn = dict(
    hidden_layer_sizes = [(128,64,16), (64,16,4)], #
    alpha=[0.01,0.02], #
    learning_rate = ['constant', 'invscaling', 'adaptive'], #
    max_iter =[1000],
    random_state=[0]
    
)

gscv_dnn = GridSearchCV(MLPClassifier(), hyper_dnn, scoring = 'accuracy', cv = n_cv)

gscv_out_dnn, score_dnn, params_dnn = hyper_tune(gscv_dnn, 'gscv_dnn', hyper_dnn, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:03:38
Accuracy: 0.8128 +/- 0.0299
optimized parameters: {'hidden_layer_sizes': (128, 64, 16), 'alpha': 0.02, 'learning_rate': 'constant', 'max_iter': 1000, 'random_state': 0}


### Ensemble voting

In [95]:
estimators=[('random forest', gscv_out_rf.best_estimator_), 
            ('logistic', gscv_out_logis.best_estimator_), 
            ('svm', gscv_out_svm.best_estimator_),
            ('"gaussian NB', gscv_out_gnb.best_estimator_),
            ('decison tree', gscv_out_dt.best_estimator_),
            ('knn', gscv_out_knn.best_estimator_),
            ('adaboost', gscv_out_adab.best_estimator_),
            ('xgboost', gscv_out_xgb.best_estimator_),
            ('catboost', gscv_out_catb.best_estimator_),
            ('dnn', gscv_out_dnn.best_estimator_)]

hyper_soft = dict(
     voting=['soft']
)

gscv_soft = GridSearchCV(VotingClassifier(estimators), hyper_soft, scoring = 'accuracy', cv = n_cv)

gscv_out_soft, score_soft, params_soft = hyper_tune(gscv_soft, 'gscv_soft', hyper_soft, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:01:09
Accuracy: 0.8303 +/- 0.0159
optimized parameters: {'voting': 'soft'}


In [103]:
hyper_stack = dict(
     meta_classifier__C= [1, 10, 100]
)

gscv_stack = GridSearchCV(StackingClassifier(classifiers = [X[1] for X in estimators],meta_classifier = LogisticRegression()),
                          hyper_stack, scoring = 'accuracy', cv = n_cv)

gscv_out_stack, score_stack, params_stack = hyper_tune(gscv_stack, 'gscv_stack', hyper_stack, n_cv, X_train_.copy(), y_train_.copy())

training time: 00:03:30
Accuracy: 0.8402 +/- 0.0341
optimized parameters: {'meta_classifier__C': 10}


## Validation


In [104]:
print("val accuracy")

print("random forest\t{:.4f}".format(accuracy_score(gscv_out_rf.best_estimator_.predict(X_val_), y_val_)))
print("logistic regr\t{:.4f}".format(accuracy_score(gscv_out_logis.best_estimator_.predict(X_val_), y_val_)))
print("svm\t\t{:.4f}".format(accuracy_score(gscv_out_svm.best_estimator_.predict(X_val_), y_val_)))
print("gaussian NB\t{:.4f}".format(accuracy_score(gscv_out_gnb.best_estimator_.predict(X_val_), y_val_)))
print("decision tree\t{:.4f}".format(accuracy_score(gscv_out_dt.best_estimator_.predict(X_val_), y_val_)))
print("knn\t\t{:.4f}".format(accuracy_score(gscv_out_knn.best_estimator_.predict(X_val_), y_val_)))
print("adaboost\t{:.4f}".format(accuracy_score(gscv_out_adab.best_estimator_.predict(X_val_), y_val_)))
print("xgboost\t\t{:.4f}".format(accuracy_score(gscv_out_xgb.best_estimator_.predict(X_val_), y_val_)))
print("catboost\t{:.4f}".format(accuracy_score(gscv_out_catb.best_estimator_.predict(X_val_), y_val_)))
print("dnn\t\t{:.4f}".format(accuracy_score(gscv_out_dnn.best_estimator_.predict(X_val_), y_val_)))
print("soft voting\t{:.4f}".format(accuracy_score(gscv_out_soft.best_estimator_.predict(X_val_), y_val_)))
print("stacking\t{:.4f}".format(accuracy_score(gscv_out_stack.best_estimator_.predict(X_val_), y_val_)))


val accuracy
random forest	0.8111
logistic regr	0.8000
svm		0.8222
gaussian NB	0.8000
decision tree	0.8333
knn		0.7889
adaboost	0.8111
xgboost		0.8444
catboost	0.8333
dnn		0.7889
soft voting	0.8222
stacking	0.8444


## Final training and prediction

In [105]:
soft_estimator = gscv_out_soft.best_estimator_.fit(X_train, y_train)
soft_pred= soft_estimator.predict(X_test).astype(int)

In [106]:
stack_estimator = gscv_out_stack.best_estimator_.fit(X_train, y_train)
stack_pred= stack_estimator.predict(X_test).astype(int)


In [109]:
soft_output = pd.DataFrame({ 'PassengerId' : df_test['PassengerId'], 'Survived': soft_pred })
soft_output.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [110]:
stack_output = pd.DataFrame({ 'PassengerId' : df_test['PassengerId'], 'Survived': stack_pred })
stack_output.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [111]:
soft_output.to_csv('data/sub_titanic_ens_soft.csv', index = False)
stack_output.to_csv('data/sub_titanic_ens_stack.csv', index = False)

<p>kaggle score</p>

<p>soft vote: 0.76</p>

<p>stacking: 0.71</p>

<p>need further investigation</p>


<h2 id="Reference">Reference</h2>

<p><a href="https://www.kaggle.com/guidant/corvus-part-1-top-15-combining-5-models">https://www.kaggle.com/guidant/corvus-part-1-top-15-combining-5-models</a></p>

<p><a href="https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier">https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier</a></p>

<p><a href="https://setscholars.net/wp-content/uploads/2019/02/find-optimal-parameters-for-CatBoost-using-GridSearchCV-for-Classification-in-Python.html" target="_blank">https://setscholars.net/wp-content/uploads/2019/02/find-optimal-parameters-for-CatBoost-using-GridSearchCV-for-Classification-in-Python.html</a></p>

<p>&nbsp;</p>

<p>&nbsp;</p>

<p>&nbsp;</p>
